In [ ]:
pip install kivy

In [17]:
pip install -U googlemaps

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import sys
import inspect
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from kivy.app import App
from kivy.core.window import Window
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.button import Button
from kivy.uix.label import Label
from kivy.uix.camera import Camera
from geocoder import get_geolocation, reverse_geocode, get_recycling_facilities, get_epa_facilities #

[INFO   ] [Logger      ] Record log in C:\Users\e4ekhan\.kivy\logs\kivy_25-03-16_36.txt
[INFO   ] [deps        ] Successfully imported "kivy_deps.angle" 0.4.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.glew" 0.3.1
[INFO   ] [deps        ] Successfully imported "kivy_deps.sdl2" 0.8.0
[INFO   ] [Kivy        ] v2.3.1
[INFO   ] [Kivy        ] Installed at "C:\Users\e4ekhan\anaconda3\Lib\site-packages\kivy\__init__.py"
[INFO   ] [Python      ] v3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
[INFO   ] [Python      ] Interpreter at "C:\Users\e4ekhan\anaconda3\python.exe"
[INFO   ] [Logger      ] Purge log fired. Processing...
[INFO   ] [Logger      ] Purge finished!
[INFO   ] [Factory     ] 195 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil (img_ffpyplayer ignored)
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] GL

In [3]:
sys.modules['__main__'].__file__ = inspect.getfile(lambda: None)

In [5]:
categories = {
    0: "Compostable",
    1: "Ewaste",
    2: "Hazardous",
    3: "Recyclables",
    4: "Textile"
}

In [7]:
VALID_CATEGORIES = ['compostable', 'ewaste', 'hazardous', 'recyclables', 'textile'] #

In [9]:
model = keras.models.load_model('NewModel.keras')

In [11]:
class TrashRecognitionApp(App):
    def build(self):
        self.layout = BoxLayout(orientation='vertical', spacing=10, padding=20)
        
        # Camera
        self.camera = Camera(resolution=(640, 480), play=True)
        self.layout.add_widget(self.camera)

        # Capture Button
        self.capture_button = Button(
            text='Capture & Predict', 
            size_hint=(1, 0.2),
            background_color=(0.2, 0.6, 0.8, 1),
            font_size=20
        )
        self.capture_button.bind(on_press=self.capture)
        self.layout.add_widget(self.capture_button)

        # Exit Button
        self.exit_button = Button(
            text='Exit App',
            size_hint=(1, 0.2),
            background_color=(0.8, 0.2, 0.2, 1),
            font_size=18
        )
        self.exit_button.bind(on_press=self.exit_app)
        self.layout.add_widget(self.exit_button)

        # Result Label
        self.result_label = Label(
            text="Prediction: Waiting...",
            font_size=20,
            color=(1, 1, 1, 1),
            bold=True
        )
        self.layout.add_widget(self.result_label)

        # Locations Label
        self.location_label = Label(
            text="Locations...",
            font_size=20,
            color=(1, 1, 1, 1),
            bold=True
        )
        self.layout.add_widget(self.location_label)

        return self.layout

    def capture(self, instance):
        texture = self.camera.texture
        if texture:
            # Image processing
            image_data = np.frombuffer(texture.pixels, dtype=np.uint8)
            image_data = image_data.reshape((texture.height, texture.width, 4))
            image_data = cv2.cvtColor(image_data, cv2.COLOR_RGBA2BGR)

            # Prediction
            resized_image = cv2.resize(image_data, (224, 224))
            input_img = np.expand_dims(resized_image, axis=0)
            result = model.predict(input_img)
            r = np.argmax(result)

            # Update UI and save category
            category = categories[r].lower().strip()
            self.result_label.text = f"Prediction: {category}"
            
            # Write to file (ADDED)
            with open('current_category.txt', 'w') as f:
                f.write(category)
                material = category.lower()
        
            # Geolocation
            GOOGLE_API_KEY = os.getenv("MAPS_API_KEY") # Add your own keys
            geolocation_data = get_geolocation(GOOGLE_API_KEY)
            if not geolocation_data:
                self.location_label.text = "Failed to get location data"
                return
            
            lat, lon, _ = geolocation_data
        
            # Reverse geocode
            location_data = reverse_geocode(GOOGLE_API_KEY, lat, lon)
            if not location_data:
                self.location_label.text = "Failed to reverse geocode"
                return
            
            city, state, _ = location_data
            self.location_label.text = f"\n📍 Detected Location: {city}, {state}"
        
            # Get facilities
            GEOAPIFY_KEY = os.getenv("GEOAPIFY_KEY") # Add you own keys
            facilities = get_recycling_facilities(GEOAPIFY_KEY, material, lat, lon)
            
            # Fallback to EPA
            if not facilities and state:
                facilities = get_epa_facilities(material, state)
            
            # Display results
            if facilities:
                self.location_label.text = f"\n♻️  Found {len(facilities)} {material} recycling options:"
                for idx, fac in enumerate(facilities[:5], 1):
                    self.location_label.text = f"{idx}. {fac['name']}"
                    self.location_label.text = f"   📍 {fac['address']}"
                    if isinstance(fac.get('distance'), (int, float)) and fac['distance'] > 0:
                        self.location_label.text = f"   📏 {round(fac['distance']/1000, 1)} km away"
            else:
                self.location_label.text = "\n⚠️  No facilities found for this material/location"

    def exit_app(self, instance):
        if self.camera.play:
            self.camera.play = False
            
        Window.close()            
        App.get_running_app().stop()

In [13]:
if __name__ == '__main__':
    TrashRecognitionApp().run()

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Geoapify API Status Code: 200
Number of facilities found: 15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
Geoapify API Status Code: 200
Number of facilities found: 15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
Geoapify API Status Code: 200
Number of facilities found: 15
